In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [4]:
from pathlib import Path
path = Path.cwd()
data_path = path.parent
data_path = str(data_path)
data_path

'/Users/keumhyeonjun/Projects/행안부/서비스용'

# 차량 데이터 불러오기 및 전처리

# 다래데이터만 > 다래 데이터 형식으로 테이블 맞춰서 제공한다고 하였음

In [ ]:
import pandas as pd

In [7]:
darae_in = pd.read_csv(data_path+ '/raw_data/VW_InCarinfo (차량 입차 정보 조회).csv')
darae_in = darae_in.dropna()
darae_out = pd.read_csv(data_path+'/raw_data/VW_OutCarinfo (차량 출차 정보 조회).csv')

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_96085/1503700987.py:1: DtypeWarning: Columns (0,1,2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  darae_in = pd.read_csv(data_path+ '/raw_data/VW_InCarinfo (차량 입차 정보 조회).csv')


In [8]:
darae_in[['입차고객구분']]= darae_in[['입차고객구분']].apply(lambda x: x.str.strip(), axis = 1)
darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']] = darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)
darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']]= darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)

## 이름 바꾸기 13_1동 지하 1 > 13

In [9]:
import re
def slicing(df,i) :
    pattern = '.*동'
    reg = re.search(pattern, df['장비명'][i])
    reg = reg.group()
    df['주차장'][i] = reg
    return reg

darae_in['주차장'] = 0
for i in range(len(darae_in)):
    slicing(darae_in,i)
darae_out['주차장'] = 0

for i in range(len(darae_out)):
    slicing(darae_out,i)

def change(x):
    if x =='13-1동' or x =='13-2동' or x =='13-3동':
        return '13동'
    elif x == '14-1동' or x == '14-2동':
        return '14동'
    else :
        return x
darae_in['주차장'] = darae_in['주차장'].apply(change)
darae_out['주차장'] = darae_out['주차장'].apply(change)

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_96085/178002389.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_96085/178002389.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg


In [10]:
pd.options.display.float_format = '{:.0f}'.format

In [11]:
import copy

In [12]:
test_in = copy.deepcopy(darae_in)
test_in.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_in['입출차여부'] = '입차'

In [13]:
test_out = copy.deepcopy(darae_out)
test_out.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_out['입출차여부'] = '출차'
test_out[test_out['장비명'].str.contains('입구')]['입출차카드번호'].value_counts()

100000000000000    15818
0                     35
Name: 입출차카드번호, dtype: int64

In [14]:
df = pd.concat([test_in, test_out]).sort_values(by=['차량번호','일자','시각'])#[['차량번호','입출차여부','일자','시각','입출차구분','출차여부','장비명']]

In [15]:
import datetime
df['입출차시각'] = df['일자'] + df['시각']
df['입출차시각'] = pd.to_datetime(df['입출차시각'], format= '%Y-%m-%d %H:%M:%S', errors='raise')
df['시'] = df['입출차시각'].dt.hour
df.reset_index(inplace = True)

In [16]:
df

,index,일자,시각,차량번호,주차일련번호,입출차카드번호,장비명,입출차구분,입출차고객구분,출차여부,차량종류,주차장,입출차여부,입출차시각,시
0,4218,2022-09-05,09:07:56,01거5868,11111220905090756,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 09:07:56,9
1,2114,2022-09-05,09:32:50,01거5868,11111220905090750,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 09:32:50,9
2,4350,2022-09-05,17:00:27,01거5868,11111220905170028,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 17:00:27,17
3,2588,2022-09-05,17:27:22,01거5868,11111220905170020,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 17:27:22,17
4,4554,2022-09-06,09:02:53,01거5868,11111220906090252,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-06 09:02:53,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77339,22206,2022-10-05,15:29:16,충북80배1724,11112221005150810,100000000039079,13-1동출구#1,정기권출차,정기권,출차,소형,13동,출차,2022-10-05 15:29:16,15
77340,9973,2022-09-10,16:07:19,충북86자3167,11112220910160720,100000000051701,13-1동입구,정상,정기권,강제출차,소형,13동,입차,2022-09-10 16:07:19,16
77341,30326,2022-09-28,17:26:49,충북86자3167,11113220928172648,100000000051701,13-2동입구,중복,정기권,출차,소형,13동,입차,2022-09-28 17:26:49,17
77342,30326,2022-09-28,17:26:49,충북86자3167,11113200000000000,100000000000000,13-2동입구,중복,정기권,출차,소형,13동,출차,2022-09-28 17:26:49,17


# 입차시간 == 출차시간인 경우 파악하기
- 입차시간 == 출차시간인 경우 입차로간주하고 출차 제거하기

In [17]:
car_arr = np.array(df[['일자','시각','차량번호']])
car_arr

array([['2022-09-05', ' 09:07:56', '01거5868'],
       ['2022-09-05', ' 09:32:50 ', '01거5868'],
       ['2022-09-05', ' 17:00:27', '01거5868'],
       ...,
       ['2022-09-28', ' 17:26:49', '충북86자3167'],
       ['2022-09-28', ' 17:26:49', '충북86자3167'],
       ['2022-09-28', ' 17:54:17 ', '충북86자3167']], dtype=object)

In [18]:
test_idx_list =[]
for idx in range(len(car_arr)-1):
    if car_arr[idx][0] == car_arr[idx+1][0] and car_arr[idx][1] == car_arr[idx+1][1] and car_arr[idx][2] == car_arr[idx+1][2]:
        test_idx_list.append(idx)
        test_idx_list.append(idx+1)
        #test_idx_list.append(idx+2)
test_idx_list = list(set(test_idx_list))
test_df = df.loc[test_idx_list]
eleminate_index = test_df[test_df['입출차여부']=='출차'].index
df2 = df.drop(eleminate_index)
df2.drop(['index'],axis = 1 , inplace = True)
df2.reset_index(inplace=True)
df2.drop(['index'],axis = 1 , inplace = True)
df2

,일자,시각,차량번호,주차일련번호,입출차카드번호,장비명,입출차구분,입출차고객구분,출차여부,차량종류,주차장,입출차여부,입출차시각,시
0,2022-09-05,09:07:56,01거5868,11111220905090756,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 09:07:56,9
1,2022-09-05,09:32:50,01거5868,11111220905090750,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 09:32:50,9
2,2022-09-05,17:00:27,01거5868,11111220905170028,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 17:00:27,17
3,2022-09-05,17:27:22,01거5868,11111220905170020,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 17:27:22,17
4,2022-09-06,09:02:53,01거5868,11111220906090252,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-06 09:02:53,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61486,2022-10-05,15:08:15,충북80배1724,11112221005150816,100000000039079,13-1동입구,정상,정기권,출차,소형,13동,입차,2022-10-05 15:08:15,15
61487,2022-10-05,15:29:16,충북80배1724,11112221005150810,100000000039079,13-1동출구#1,정기권출차,정기권,출차,소형,13동,출차,2022-10-05 15:29:16,15
61488,2022-09-10,16:07:19,충북86자3167,11112220910160720,100000000051701,13-1동입구,정상,정기권,강제출차,소형,13동,입차,2022-09-10 16:07:19,16
61489,2022-09-28,17:26:49,충북86자3167,11113220928172648,100000000051701,13-2동입구,중복,정기권,출차,소형,13동,입차,2022-09-28 17:26:49,17


# 데이터 전처리
- 아침시간이니 입차시간 == 출차시간인 경우 입차로 가정
- 차량 번호별로 입차1,입차2,.....출차1,출차2 인경우  >>  입차1, 출차
- 입차1, 입차2, ......

In [19]:
car_arr = np.array(df2[['차량번호','입출차여부','시각']])
idx_list = []
for idx in range(len(car_arr)):
    if idx == 0 :
        idx_list.append(idx)
    else :  # 차량 번호가 같고 상태가 다른 경우(입차 > 출차 or 출차 > 입차)
        if (car_arr[idx][0] == car_arr[idx-1][0]) and (car_arr[idx][1]!=car_arr[idx-1][1]):
            idx_list.append(idx)

            #차량번호가 바뀌는 경우
        elif car_arr[idx][0] != car_arr[idx-1][0]:
            idx_list.append(idx)



In [20]:
process_df = df2.loc[idx_list]
process_df[process_df['차량번호'] == '85부3062']

,일자,시각,차량번호,주차일련번호,입출차카드번호,장비명,입출차구분,입출차고객구분,출차여부,차량종류,주차장,입출차여부,입출차시각,시
58299,2022-09-01,07:17:42,85부3062,11111220901071742,100000000024581,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-01 07:17:42,7
58300,2022-09-01,07:27:23,85부3062,12112220831154730,100000000024581,13-1동출구#1,정기권출차,정기권,출차,소형,13동,출차,2022-09-01 07:27:23,7
58301,2022-09-01,07:28:59,85부3062,11114220901072860,100000000024581,13-3동입구,정상,정기권,출차,소형,13동,입차,2022-09-01 07:28:59,7
58302,2022-09-01,07:35:58,85부3062,11111220901071740,100000000024581,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-01 07:35:58,7
58303,2022-09-01,07:38:43,85부3062,11112220901073844,100000000024581,13-1동입구,정상,정기권,강제출차,소형,13동,입차,2022-09-01 07:38:43,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58679,2022-10-05,08:10:00,85부3062,11112221005073210,100000000024581,13-1동출구#1,정기권출차,정기권,출차,소형,13동,출차,2022-10-05 08:10:00,8
58680,2022-10-05,08:12:25,85부3062,11112221005081224,100000000024581,13-1동입구,정상,정기권,출차,소형,13동,입차,2022-10-05 08:12:25,8
58681,2022-10-05,08:18:22,85부3062,14111221005075710,100000000024581,14-2동출구#1,정기권출차,정기권,출차,소형,14동,출차,2022-10-05 08:18:22,8
58683,2022-10-05,15:33:40,85부3062,11112221005153340,100000000024581,13-1동입구,정상,정기권,출차,소형,13동,입차,2022-10-05 15:33:40,15


In [21]:
process_df = df2.loc[idx_list]
process_df['입출차여부'].value_counts()

process_df['연'] = process_df['입출차시각'].dt.year
process_df['월'] = process_df['입출차시각'].dt.month
process_df['일'] = process_df['입출차시각'].dt.day
process_df['시'] = process_df['입출차시각'].dt.hour
process_df['분'] = process_df['입출차시각'].dt.minute
process_df['초'] = process_df['입출차시각'].dt.second
process_df = process_df[(process_df['시']>=7) & (process_df['시']<=9)]
process_df

,일자,시각,차량번호,주차일련번호,입출차카드번호,장비명,입출차구분,입출차고객구분,출차여부,차량종류,주차장,입출차여부,입출차시각,시,연,월,일,분,초
0,2022-09-05,09:07:56,01거5868,11111220905090756,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 09:07:56,9,2022,9,5,7,56
1,2022-09-05,09:32:50,01거5868,11111220905090750,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 09:32:50,9,2022,9,5,32,50
4,2022-09-06,09:02:53,01거5868,11111220906090252,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-06 09:02:53,9,2022,9,6,2,53
5,2022-09-06,09:27:11,01거5868,11111220906090250,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-06 09:27:11,9,2022,9,6,27,11
6,2022-10-04,09:02:20,01거5868,11111221004090220,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-10-04 09:02:20,9,2022,10,4,2,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61401,2022-09-19,09:29:29,충북80배1724,11111220919091030,100000000039079,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-19 09:29:29,9,2022,9,19,29,29
61402,2022-09-19,09:33:42,충북80배1724,11112220919093342,100000000039079,13-1동입구,정상,정기권,출차,소형,13동,입차,2022-09-19 09:33:42,9,2022,9,19,33,42
61403,2022-09-19,09:54:15,충북80배1724,11112220919093340,100000000039079,13-1동출구#1,정기권출차,정기권,출차,소형,13동,출차,2022-09-19 09:54:15,9,2022,9,19,54,15
61404,2022-09-20,09:11:59,충북80배1724,11112220920091160,100000000039079,13-1동입구,정상,정기권,출차,소형,13동,입차,2022-09-20 09:11:59,9,2022,9,20,11,59


In [22]:
test = process_df[process_df['일']==15]
test[test['주차장']=='13동']['입출차여부'].value_counts()

입차    133
출차     15
Name: 입출차여부, dtype: int64

In [24]:
parking_info = pd.read_excel(data_path + '/raw_data/주차장면수.xlsx')
parking_info

,구분,장소,합계,지상,지하
0,청사내부,1동,304,100,204
1,청사내부,2동,177,73,104
2,청사내부,3동,26,26,0
3,청사내부,4동,191,138,53
4,청사내부,5동,190,81,109
5,청사내부,6동,616,107,509
6,청사내부,7동,168,33,135
7,청사내부,8동,146,42,104
8,청사내부,9동,82,15,67
9,청사내부,10동,123,32,91


# 데이터셋 생성

In [25]:
dataset = pd.merge(process_df,parking_info[['장소','합계']], left_on='주차장', right_on='장소')

In [26]:
dataset = dataset[['입출차시각','연','월','일','시','분','초','입출차여부','주차장','합계']]

In [27]:
dataset = dataset.sort_values(by = ['주차장','입출차시각'])
dataset['잔류차량수'] = 0

In [28]:
dataset

,입출차시각,연,월,일,시,분,초,입출차여부,주차장,합계,잔류차량수
532,2022-09-01 07:07:31,2022,9,1,7,7,31,입차,12동,82,0
4679,2022-09-01 07:17:42,2022,9,1,7,17,42,입차,12동,82,0
4569,2022-09-01 07:18:36,2022,9,1,7,18,36,입차,12동,82,0
2015,2022-09-01 07:35:54,2022,9,1,7,35,54,입차,12동,82,0
4680,2022-09-01 07:35:58,2022,9,1,7,35,58,출차,12동,82,0
...,...,...,...,...,...,...,...,...,...,...,...
8780,2022-10-05 09:10:01,2022,10,5,9,10,1,입차,15동,201,0
8340,2022-10-05 09:20:45,2022,10,5,9,20,45,입차,15동,201,0
8392,2022-10-05 09:32:20,2022,10,5,9,32,20,출차,15동,201,0
9411,2022-10-05 09:48:30,2022,10,5,9,48,30,출차,15동,201,0


In [29]:
arr = np.array(dataset[['연','월','일','시','분','입출차여부','주차장','잔류차량수']])
arr

array([[2022, 9, 1, ..., '입차', '12동', 0],
       [2022, 9, 1, ..., '입차', '12동', 0],
       [2022, 9, 1, ..., '입차', '12동', 0],
       ...,
       [2022, 10, 5, ..., '출차', '15동', 0],
       [2022, 10, 5, ..., '출차', '15동', 0],
       [2022, 10, 5, ..., '입차', '15동', 0]], dtype=object)

In [30]:
for idx in range(len(arr)):
    if idx == 0 :
        if arr[idx][5] == '입차' :
            arr[idx][7] = arr[idx][7] +1
        else :
            arr[idx][7] = arr[idx][7] -1
    else :
        pre_time = list(arr[idx-1][[0,1,2,6]])
        now_time = list(arr[idx][[0,1,2,6]])
        status =  arr[idx][5]
            # 이전 데이터와 연,월,일 주차장이 같은 경우
        if pre_time == now_time :
            #print('>')
            if status == '입차' :
                arr[idx][7] = arr[idx-1][7] +1
            else :
                arr[idx][7] = arr[idx-1][7] -1

            #이전 데이터와 연,월,일 주차장이 다른 경우 > 0부터 시작함
        else :
            if status == '입차':
                arr[idx][7] =1
            else :
                arr[idx][7] = -1


In [31]:
dataset[['연','월','일','시','분','입출차여부','주차장','잔류차량수']] = arr
dataset

,입출차시각,연,월,일,시,분,초,입출차여부,주차장,합계,잔류차량수
532,2022-09-01 07:07:31,2022,9,1,7,7,31,입차,12동,82,1
4679,2022-09-01 07:17:42,2022,9,1,7,17,42,입차,12동,82,2
4569,2022-09-01 07:18:36,2022,9,1,7,18,36,입차,12동,82,3
2015,2022-09-01 07:35:54,2022,9,1,7,35,54,입차,12동,82,4
4680,2022-09-01 07:35:58,2022,9,1,7,35,58,출차,12동,82,3
...,...,...,...,...,...,...,...,...,...,...,...
8780,2022-10-05 09:10:01,2022,10,5,9,10,1,입차,15동,201,75
8340,2022-10-05 09:20:45,2022,10,5,9,20,45,입차,15동,201,76
8392,2022-10-05 09:32:20,2022,10,5,9,32,20,출차,15동,201,75
9411,2022-10-05 09:48:30,2022,10,5,9,48,30,출차,15동,201,74


# 혼잡도 만들기

In [32]:
pd.options.display.float_format = '{:.5f}'.format
dataset['비율']=dataset['잔류차량수']/dataset['합계']

def confusion(df):
    if df['비율'] <= 0.3 :
        return 0
    elif df['비율'] <= 0.5 :
        return 1
    elif df['비율'] <= 0.8 :
        return 2
    else :
        return 3

dataset['혼잡도']=dataset.apply(confusion,axis=1)
dataset

,입출차시각,연,월,일,시,분,초,입출차여부,주차장,합계,잔류차량수,비율,혼잡도
532,2022-09-01 07:07:31,2022,9,1,7,7,31,입차,12동,82,1,0.01220,0
4679,2022-09-01 07:17:42,2022,9,1,7,17,42,입차,12동,82,2,0.02439,0
4569,2022-09-01 07:18:36,2022,9,1,7,18,36,입차,12동,82,3,0.03659,0
2015,2022-09-01 07:35:54,2022,9,1,7,35,54,입차,12동,82,4,0.04878,0
4680,2022-09-01 07:35:58,2022,9,1,7,35,58,출차,12동,82,3,0.03659,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8780,2022-10-05 09:10:01,2022,10,5,9,10,1,입차,15동,201,75,0.37313,1
8340,2022-10-05 09:20:45,2022,10,5,9,20,45,입차,15동,201,76,0.37811,1
8392,2022-10-05 09:32:20,2022,10,5,9,32,20,출차,15동,201,75,0.37313,1
9411,2022-10-05 09:48:30,2022,10,5,9,48,30,출차,15동,201,74,0.36816,1


# 날씨데이터 붙이기

In [34]:
# whether csv read, 일시를 index로 넣고 datetime 형태로 변환함
whether = pd.read_csv(data_path +'/data/Whether.csv', encoding='euc-kr', parse_dates=['일시'], index_col='일시')
# 강수량, 적설이 null인 경우 모두 0으로 바꿈
whether = whether.fillna(0)
# 일시의 freq을 hour로 변경
whether.index.freq = 'h'

'''
데이터가 누락된 시간을 확인하기 위한 부분 (파일로 받은 경우)
API로 받는 경우 날짜는 있지만 모든 데이터가 Null값이기 때문에 해당 조건으로 수정해야 할 것으로 보임

j = whether.index[0]
for i in whether.index:
    if i != j + pd.Timedelta("1 hour"):
        print(j, i)
    j = i
'''
# 지점코드, 지점명을 없앰
whether = whether.iloc[:, [2,3,4,5,6]]
whether


whether = whether.reset_index()
whether['연'] = whether['일시'].dt.year
whether['월'] = whether['일시'].dt.month
whether['일'] = whether['일시'].dt.day
whether['시'] = whether['일시'].dt.hour
whether

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),연,월,일,시
0,2020-10-01 00:00:00,17.30000,0.00000,0.20000,92.00000,0.00000,2020,10,1,0
1,2020-10-01 01:00:00,16.80000,0.00000,0.50000,94.00000,0.00000,2020,10,1,1
2,2020-10-01 02:00:00,16.90000,0.00000,1.10000,93.00000,0.00000,2020,10,1,2
3,2020-10-01 03:00:00,16.80000,0.00000,0.40000,94.00000,0.00000,2020,10,1,3
4,2020-10-01 04:00:00,16.10000,0.00000,0.10000,95.00000,0.00000,2020,10,1,4
...,...,...,...,...,...,...,...,...,...,...
17539,2022-10-01 19:00:00,21.10000,0.00000,0.10000,54.00000,0.00000,2022,10,1,19
17540,2022-10-01 20:00:00,19.60000,0.00000,0.60000,63.00000,0.00000,2022,10,1,20
17541,2022-10-01 21:00:00,18.80000,0.00000,0.70000,73.00000,0.00000,2022,10,1,21
17542,2022-10-01 22:00:00,18.10000,0.00000,0.80000,75.00000,0.00000,2022,10,1,22


In [35]:
dataset = pd.merge(dataset,whether, on =['연','월','일','시'], how= 'left')
dataset

,입출차시각,연,월,일,시,분,초,입출차여부,주차장,합계,잔류차량수,비율,혼잡도,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
0,2022-09-01 07:07:31,2022,9,1,7,7,31,입차,12동,82,1,0.01220,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000
1,2022-09-01 07:17:42,2022,9,1,7,17,42,입차,12동,82,2,0.02439,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000
2,2022-09-01 07:18:36,2022,9,1,7,18,36,입차,12동,82,3,0.03659,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000
3,2022-09-01 07:35:54,2022,9,1,7,35,54,입차,12동,82,4,0.04878,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000
4,2022-09-01 07:35:58,2022,9,1,7,35,58,출차,12동,82,3,0.03659,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15724,2022-10-05 09:10:01,2022,10,5,9,10,1,입차,15동,201,75,0.37313,1,NaT,NaN,NaN,NaN,NaN,NaN
15725,2022-10-05 09:20:45,2022,10,5,9,20,45,입차,15동,201,76,0.37811,1,NaT,NaN,NaN,NaN,NaN,NaN
15726,2022-10-05 09:32:20,2022,10,5,9,32,20,출차,15동,201,75,0.37313,1,NaT,NaN,NaN,NaN,NaN,NaN
15727,2022-10-05 09:48:30,2022,10,5,9,48,30,출차,15동,201,74,0.36816,1,NaT,NaN,NaN,NaN,NaN,NaN


In [36]:
import holidays
kr_holidays = holidays.KR()
def holiday_check(x):
    if x in kr_holidays:
        return 1
    else :
        return 0


dataset['공휴일'] = dataset['입출차시각'].apply(holiday_check)
dataset['요일'] = dataset['입출차시각'].dt.weekday
dataset

,입출차시각,연,월,일,시,분,초,입출차여부,주차장,합계,...,비율,혼잡도,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),공휴일,요일
0,2022-09-01 07:07:31,2022,9,1,7,7,31,입차,12동,82,...,0.01220,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000,0,3
1,2022-09-01 07:17:42,2022,9,1,7,17,42,입차,12동,82,...,0.02439,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000,0,3
2,2022-09-01 07:18:36,2022,9,1,7,18,36,입차,12동,82,...,0.03659,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000,0,3
3,2022-09-01 07:35:54,2022,9,1,7,35,54,입차,12동,82,...,0.04878,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000,0,3
4,2022-09-01 07:35:58,2022,9,1,7,35,58,출차,12동,82,...,0.03659,0,2022-09-01 07:00:00,20.20000,0.00000,0.90000,94.00000,0.00000,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15724,2022-10-05 09:10:01,2022,10,5,9,10,1,입차,15동,201,...,0.37313,1,NaT,NaN,NaN,NaN,NaN,NaN,0,2
15725,2022-10-05 09:20:45,2022,10,5,9,20,45,입차,15동,201,...,0.37811,1,NaT,NaN,NaN,NaN,NaN,NaN,0,2
15726,2022-10-05 09:32:20,2022,10,5,9,32,20,출차,15동,201,...,0.37313,1,NaT,NaN,NaN,NaN,NaN,NaN,0,2
15727,2022-10-05 09:48:30,2022,10,5,9,48,30,출차,15동,201,...,0.36816,1,NaT,NaN,NaN,NaN,NaN,NaN,0,2


In [37]:
dataset = dataset[['입출차시각', '연', '월', '일', '시', '분','초','공휴일','요일','주차장',
         '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)',
         '잔류차량수', '비율','혼잡도']]
# 날짜 매칭이 안되는 데이터들 제거
dataset = dataset.dropna()

In [41]:
dataset.to_csv(data_path + '/data/dataset_darae.csv')

# 보정하기
 - 잔류차량수 >주차면수 인 경우 어떻게??
-  생각해봐야함

In [ ]:
a = dataset.groupby(['연','월','일','주차장'])['잔류차량수','합계'].max()
a=pd.DataFrame(a)
a.reset_index(inplace=True)

In [ ]:
a['전체-잔류'] = a['합계'] - a['잔류차량수']
a